In [1]:
!cd

E:\Projets\Dog and cat


In [1]:
!kaggle competitions download -c dogs-vs-cats -p Data/

dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [1]:
import os
import shutil
import zipfile
import matplotlib.image as image
from io import BytesIO
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.layers import Conv2D,MaxPooling2D,Activation,Dense,Flatten,Dropout
from keras.models import Sequential
from keras.initializers import RandomNormal,RandomUniform
from keras.optimizers import Adam,RMSprop
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [2]:
print('Extracting files ....')
with zipfile.ZipFile('Data/dogs-vs-cats.zip') as zfiles:
    for file in zfiles.namelist():
        try:
            zfiledata = BytesIO(zfiles.read(file))
            efile = zipfile.ZipFile(zfiledata)
            efile.extractall('data/')
        except:
            print('An error occured')
print('Extraction completed ....')
print('Extracted file location (data/).... ')

Extracting files ....
An error occured
Extraction completed ....
Extracted file location (data/).... 


In [3]:
train_data_path = './Data/train/'
test_data_path = './Data/test1/'

In [4]:
def get_data_ready(path):
    print('Loading Files .... ')
    print('Please Wait ....')
    label, file_list ,=[],[]
    #data = []
    for files in os.walk(path):
        for file in files[2]:
            filename, extension = os.path.splitext(file)
            if extension == '.jpg':
                #load_path = path+file
                #img = image.imread(load_path)
                #data.append(img)
                if len(filename.split('.'))>1:
                    file_list.append(file)
                    label.append(file.split('.')[0])
                else:
                    file_list.append(file)
            else:
                print('Not a .jpg file  the actual extension is {}'.format(extension))
                
#    data = np.asarray(data)
    label = np.asarray(label)
    print('Files Loading Complete ....')
    return label,file_list

In [5]:
train_labels,train_filename = get_data_ready(train_data_path)

Loading Files .... 
Please Wait ....
Files Loading Complete ....


In [6]:
test_labels,test_filename = get_data_ready(test_data_path)

Loading Files .... 
Please Wait ....
Files Loading Complete ....


In [7]:
train_df = pd.DataFrame(data=zip(train_filename,train_labels),columns=['filename','labels'],)
test_df = pd.DataFrame(data=test_filename,columns=['filename'])

In [55]:
train_data,valid_data = train_test_split(train_df,
                                         random_state=42,
                                         test_size=0.25,
                                         shuffle=True)

In [56]:
data_gen =ImageDataGenerator(zoom_range=0.2,
                             rescale=(1./255.),
                             horizontal_flip=True,
                             shear_range=0.2)

In [10]:
valid_data.reset_index(drop=True,inplace=True)
train_data.reset_index(drop=True,inplace=True)

In [58]:
train_data_generator = data_gen.flow_from_dataframe(dataframe = train_data,
                                                    shuffle=True,
                                                    directory = './Data/train/',
                                                    x_col = 'filename',
                                                    y_col = 'labels',
                                                    target_size = (128,128),
                                                    batch_size = 32,
                                                    class_mode = 'binary')

valid_data_generator = data_gen.flow_from_dataframe(dataframe = valid_data,
                                                    shuffle=True,
                                                    directory = 'Data/train/',
                                                    x_col = 'filename',
                                                    y_col = 'labels',
                                                    target_size = (128,128),
                                                    batch_size = 32,
                                                    class_mode = 'binary')

Found 18750 validated image filenames belonging to 2 classes.
Found 6250 validated image filenames belonging to 2 classes.


In [59]:
l =[]
for i in (valid_data_generator.filenames):
    l.append(i.split('.')[0])

In [60]:
np.unique(l)

array(['cat', 'dog'], dtype='<U3')

In [61]:
train_data['filename'].nunique()

18750

In [62]:
model1 = Sequential()

model1.add(Conv2D(filters = 32 ,
                  kernel_size = (3,3),
                  strides = (2,2),
                  padding = 'same',
                  activation='relu',
                  input_shape = (128,128,3)))

model1.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1),
                        padding='valid'))

model1.add(Dropout(0.25))

model1.add(Conv2D(filters = 64 ,
                  kernel_size = (3,3),
                  strides = (1,1),
                  padding = 'valid',
                  activation='relu'))

model1.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1), 
                        padding='valid'))

model1.add(Conv2D(filters = 64 ,
                  kernel_size = (3,3),
                  strides = (2,2),
                  padding = 'same',
                  activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1),
                        padding='valid'))

model1.add(Dropout(0.25))
model1.add(Conv2D(filters = 64 ,
                  kernel_size = (3,3),
                  strides = (1,1),
                  padding = 'valid',
                  activation='relu'))

model1.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1),
                        padding='valid'))


model1.add(Conv2D(filters = 128 ,
                  kernel_size = (3,3),
                  strides = (2,2),
                  padding = 'same',
                  activation='relu'))

model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(units=64,activation = 'relu'))
model1.add(Dense(units=128,activation = 'relu'))
model1.add(Dense(units=256,activation = 'relu'))
model1.add(Dense(units=1,activation = 'sigmoid'))


model1.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 60, 60, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 30, 30, 64)        36928     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 29, 29, 64)       

In [63]:
opt = Adam(learning_rate= 0.1)
model1.compile(optimizer=opt,
               loss = 'binary_crossentropy',
               metrics = 'accuracy')

In [64]:
model1_summary = model1.fit(train_data_generator,
                            shuffle=True,
                            epochs = 1,
                            validation_data=valid_data_generator)

586/586 [==============================] - 468s 798ms/step - loss: 48206.1328 - accuracy: 0.4959 - val_loss: 0.6934 - val_accuracy: 0.5026


In [68]:
model2 = Sequential()

model2.add(Conv2D(filters = 32 ,
                  kernel_size = (3,3),
                  strides = (2,2),
                  padding = 'same',
                  activation='relu',
                  input_shape = (128,128,3)))

model2.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1),
                        padding='valid'))

model2.add(Dropout(0.25))

model2.add(Conv2D(filters = 64 ,
                  kernel_size = (3,3),
                  strides = (1,1),
                  padding = 'valid',
                  activation='relu'))

model2.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1), 
                        padding='valid'))

model2.add(Conv2D(filters = 128 ,
                  kernel_size = (3,3),
                  strides = (2,2),
                  padding = 'same',
                  activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1),
                        padding='valid'))

model2.add(Dropout(0.25))
model2.add(Conv2D(filters = 256 ,
                  kernel_size = (3,3),
                  strides = (1,1),
                  padding = 'valid',
                  activation='relu'))

model2.add(MaxPooling2D(pool_size=(2,2),
                        strides=(1, 1),
                        padding='valid'))


model2.add(Conv2D(filters = 256 ,
                  kernel_size = (3,3),
                  strides = (2,2),
                  padding = 'same',
                  activation='relu'))

model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(units=64,activation = 'relu'))
model2.add(Dense(units=128,activation = 'relu'))
model2.add(Dense(units=256,activation = 'relu'))
model2.add(Dense(units=1,activation = 'sigmoid'))


model2.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 60, 60, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 30, 30, 128)       73856     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 29, 29, 128)      

In [69]:
opt = Adam(learning_rate= 0.1)
model2.compile(optimizer=opt,
               loss = 'binary_crossentropy',
               metrics = 'accuracy')

In [70]:
model2_summary = model2.fit(train_data_generator,
                            shuffle=True,
                            epochs = 10,
                            validation_data=valid_data_generator)

Epoch 1/10
586/586 [==============================] - 773s 1s/step - loss: 9912732.0000 - accuracy: 0.5042 - val_loss: 0.7150 - val_accuracy: 0.4974
Epoch 2/10
586/586 [==============================] - 668s 1s/step - loss: 0.7070 - accuracy: 0.4950 - val_loss: 0.6933 - val_accuracy: 0.4974
Epoch 3/10
586/586 [==============================] - 721s 1s/step - loss: 0.6962 - accuracy: 0.4981 - val_loss: 0.6934 - val_accuracy: 0.4974
Epoch 4/10
586/586 [==============================] - 697s 1s/step - loss: 0.6951 - accuracy: 0.4969 - val_loss: 0.6936 - val_accuracy: 0.4974
Epoch 5/10
586/586 [==============================] - 697s 1s/step - loss: 0.6953 - accuracy: 0.4936 - val_loss: 0.6931 - val_accuracy: 0.5026
Epoch 6/10
586/586 [==============================] - 612s 1s/step - loss: 0.6949 - accuracy: 0.5059 - val_loss: 0.6936 - val_accuracy: 0.5026
Epoch 7/10
586/586 [==============================] - 638s 1s/step - loss: 0.6948 - accuracy: 0.5037 - val_loss: 0.6938 - val_accuracy: 

In [32]:
test_gen = ImageDataGenerator(rescale=(1./255.))
test_data_generator = test_gen.flow_from_dataframe(test_df,
                                                   directory='./Data/test1/',
                                                   x_col='filename',
                                                   y_col=None,
                                                   class_mode=None,
                                                   target_size=(128,128),
                                                   batch_size=32)

Found 12500 validated image filenames.


In [65]:
pred1 = model1.predict(test_data_generator)

In [40]:
train_data_generator.class_indices

{'cat': 0, 'dog': 1}

In [ ]:
pred2 = model2.predict(test_data_generator)

In [34]:
pred1

array([[0.6681187],
       [0.6681187],
       [0.6681187],
       ...,
       [0.6681187],
       [0.6681187],
       [0.6681187]], dtype=float32)

In [66]:
test_labels = []
for i in pred1:
    if i>0.5:
        test_labels.append('dog')
    else:
        test_labels.append('cat')

In [67]:
np.unique(test_labels)

array(['dog'], dtype='<U3')

In [78]:
model = Sequential()

model.add(Conv2D(64, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block1_conv1', input_shape=(128, 128, 3)))
model.add(Conv2D(64, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block1_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block1_maxpool'))

model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block2_conv1'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block2_maxpool'))

model.add(Conv2D(256, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv1'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block3_maxpool'))

model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv1'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block4_maxpool'))

model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv1'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block5_maxpool'))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_maxpool (MaxPooling2D (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_maxpool (MaxPooling2D (None, 32, 32, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 32, 32, 256)       295168    
_________________________________________________________________
block3_maxpool (MaxPooling2D (None, 16, 16, 256)     

In [79]:
opt = Adam(learning_rate= 0.1)
model.compile(optimizer=opt,
               loss = 'binary_crossentropy',
               metrics = 'accuracy')

In [ ]:
model_summary = model.fit(train_data_generator,
                            shuffle=True,
                            epochs = 10,
                            validation_data=valid_data_generator)

Epoch 1/10
586/586 [==============================] - 1963s 3s/step - loss: 1965393152.0000 - accuracy: 0.4969 - val_loss: 3.1591 - val_accuracy: 0.5026
Epoch 2/10
352/586 [=================>............] - ETA: 9:55 - loss: 104.4308 - accuracy: 0.4989